In [1]:
import voice_handler as vh
import os
import pandas as pd
import openai
import re
import numpy as np
openai.api_key = <OPENAI_KEY>
# openai.Model.list()

In [3]:
def query_response(s, value):
    result = re.search(value+': (.*)\n', s)
    return result.group(1)

In [4]:
def update_df(df, s):
    theme = query_response(s, 'THEME')
    detail = query_response(s, 'DETAIL')
    emotion = query_response(s, 'EMOTION')
    valence = query_response(s, 'VALENCE')
    arousal = query_response(s, 'AROUSAL')
    name = query_response(s, 'NAME')
    relationship = query_response(s, 'RELATIONSHIP')
    
    if theme in df['Theme'].values:
        idx = df.index[df['Theme'] == theme][0]
        df.at[idx, 'Occurrences'] += 1
        df.at[idx, 'Valence'] = valence
        df.at[idx, 'Arousal'] = arousal
        df.at[idx, 'Emotional State'] = emotion
        df.at[idx, 'Specific Detail'] = detail
        df.at[idx, 'Name'] = name
        df.at[idx, 'Relationship'] = relationship
    else:
        newdf = pd.DataFrame({'Theme':theme, 'Specific Detail':detail, 'Emotional State':emotion, 'Valence':valence, 'Arousal':arousal, 'Name':name, 'Relationship':relationship, 'Occurrences':1}, index=[0])
        df = pd.concat([df, newdf], ignore_index=True)
        
    return df
    

In [5]:
def GPT_talk(df, temp=0.0, freq_penalty=0.0, max_token=2048):
    sentiment_query = "1) Classify my emotional state and label it with keyword 'EMOTION': \n2) Identify a specific detail about the prompt and label with keyword 'DETAIL': \n3) Identify a theme for why I am feeling this way and label it with keyword 'THEME': \n4) Rate the valence on a scale of -1 to 1, label with keyword 'VALENCE': \n5) Rate arousal on from 0 to 1 and label with keyword 'AROUSAL': \n6) Use keyword NAME: If a person's name is mentioned, label it here, otherwise type N/A. \n7) Use keyword RELATIONSHIP: If a person's name is mentioned, identify their relationship to me otherwise label it with N/A' \n8) Label with keyword 'RESPONSE'. Make sure this response feels natural and conversational. If the valence > 0, respond by asking relevant questions. If valence < 0, comfort me. "
    
    idx = np.where(df['Valence'].astype(float)>0)[0]
    arr_themes = df['Theme'][idx].values
    arr_details = df['Specific Detail'][idx].values
    
    
    change_topic = "\nSteer the conversation towards the following topics: " + ', '.join(arr_themes) + ', ' + ', '.join(arr_details)
    
    speech = vh.speech_to_text()
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt= sentiment_query + change_topic + "\n\n" + speech["transcription"] + ".",
      temperature=temp,
      max_tokens=max_token,
      top_p=1.0,
      frequency_penalty=freq_penalty,
      presence_penalty=0.0
    )
    print(sentiment_query + change_topic + "\n\n" + speech["transcription"] + ".")
    print("\n")
    s = response["choices"][0]["text"]
    print(s)
    start_idx = s.find('RESPONSE')
    vh.text_to_speech(s[start_idx+9:])
    
#     df = update_df(df, s)
        
    return response, df

In [7]:
def GPT_listen(df, temp=0.0, freq_penalty=0.0, max_token=2048):
    sentiment_query = "1) Classify my emotional state and label it with keyword 'EMOTION': \n2) Identify a specific detail about the prompt and label with keyword 'DETAIL': \n3) Identify a theme for why I am feeling this way and label it with keyword 'THEME': \n4) Rate the valence on a scale of -1 to 1, label with keyword 'VALENCE': \n5) Rate arousal on from 0 to 1 and label with keyword 'AROUSAL': \n6) Use keyword NAME: If a person's name is mentioned, label it here, otherwise type N/A. \n7) Use keyword RELATIONSHIP: If a person's name is mentioned, identify their relationship to me otherwise label it with N/A \n8) Use keyword RESPONSE: type N/A"
        
    speech = vh.speech_to_text()
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt= sentiment_query + "\n\n" + speech["transcription"] + ".",
      temperature=temp,
      max_tokens=max_token,
      top_p=1.0,
      frequency_penalty=freq_penalty,
      presence_penalty=0.0
    )
    print("\n")
    s = response["choices"][0]["text"]
    print(s)
    
    df = update_df(df, s)
        
    return response, df

In [8]:
df = pd.DataFrame(columns=['Theme', 'Specific Detail', 'Emotional State', 'Valence', 'Arousal', 'Name', 'Relationship', 'Occurrences'])
print(df)

Empty DataFrame
Columns: [Theme, Specific Detail, Emotional State, Valence, Arousal, Name, Relationship, Occurrences]
Index: []


In [13]:
response, df = GPT_listen(df, temp=0.0, max_token=2048, freq_penalty=0.0)

print("\n\n\n\n")
print(df)

Adjusting for ambient noise...
Listening...
I have such fond memories of driving while listening to The Beatles




EMOTION: Nostalgic
DETAIL: Driving while listening to The Beatles
THEME: Memories
VALENCE: 0.8
AROUSAL: 0.6
NAME: N/A
RELATIONSHIP: N/A
RESPONSE: N/A





          Theme                         Specific Detail Emotional State  \
0  Uncertainty                          Question asked        Confused    
1   Recognition                             Nick's name          Relief   
2    Inadequacy                                 Driving     Frustration   
3      Memories  Driving while listening to The Beatles       Nostalgic   

  Valence Arousal  Name Relationship Occurrences  
0      0     0.5   N/A          N/A            1  
1     0.8     0.5  Nick       Nephew           1  
2    -0.5     0.8   N/A          N/A           1  
3     0.8     0.6   N/A          N/A           1  


In [42]:
response, df = GPT_talk(df, temp=0.0, max_token=2048, freq_penalty=0.0)

print("\n\n\n\n")
print(df)

Adjusting for ambient noise...
Listening...
I don't know why everyone is treating me like a child
1) Classify my emotional state and label it with keyword 'EMOTION': 
2) Identify a specific detail about the prompt and label with keyword 'DETAIL': 
3) Identify a theme for why I am feeling this way and label it with keyword 'THEME': 
4) Rate the valence on a scale of -1 to 1, label with keyword 'VALENCE': 
5) Rate arousal on from 0 to 1 and label with keyword 'AROUSAL': 
6) Use keyword NAME: If a person's name is mentioned, label it here, otherwise type N/A. 
7) Use keyword RELATIONSHIP: If a person's name is mentioned, identify their relationship to me otherwise label it with N/A' 
8) Label with keyword 'RESPONSE'. Make sure this response feels natural and conversational. If the valence > 0, respond by asking relevant questions. If valence < 0, comfort me. 
Steer the conversation towards the following topics: Recognition, Memories, Nick's name, Driving while listening to The Beatles

I 